In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_table("./data/abalone.data", delimiter=",", header=None)

# Convert categorical data to numerical
data[0] = pd.factorize(data[0])[0]

# Split the data into features (X) and target (y)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Convert to numpy arrays
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
# Split the data into batches


# Define Layer class
class Layer:
    def __init__(self, n_inputs, n_neurons, type = 0):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        # self.weights = np.random.randint(1, 4, (n_inputs, n_neurons))
        self.biases = np.random.randn(1, n_neurons)
        if type == 0:
            self.activation = Sigmoid()
        else:
            self.activation = ReLU()

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
        self.activation.forward(self.output)

    def backward(self, dvalues , output_layer = 0):
        # print(len(dvalues.shape))
        if len(dvalues.shape) == 1:
            dvalues = dvalues.reshape((dvalues.shape[0],1))

        # print(dvalues.shape)[0]
        # print("dvalues", dvalues, dvalues.shape)
        if output_layer:
            print("Output Layer")
        else:
            print("Hidden Layer")

        self.dinputs = np.dot(dvalues, self.weights.T)
        # print("dinputs", self.dinputs, self.dinputs.shape)
        self.dweights = np.dot(self.inputs.T, dvalues)
        # print("inputs", self.inputs, self.inputs.shape)
        # print("dweights", self.dweights, self.dweights.shape)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # print("dbiases", self.dbiases, self.dbiases.shape)
        self.activation.backward(self.dinputs)
        # print("dinputs", self.activation.dinputs, self.activation.dinputs.shape)
    def update(self, learning_rate):
        print(type(learning_rate), type(self.dweights))
        self.weights += learning_rate * self.dweights
        self.biases += learning_rate * self.dbiases


# Define ReLU activation function
class ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        print("dinputs", self.dinputs, self.dinputs.shape)
        mask = self.dinputs <= 0
        print("mask", mask, mask.shape)
        self.dinputs[self.dinputs <= 0] = 0
        print("dinputs", self.dinputs, self.dinputs.shape)
        

class Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        # self.output = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output


class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        batch_loss = np.mean(sample_losses)
        return batch_loss

class MeanSquaredError(Loss):
    def forward(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2, axis=0)
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        self.dinputs = -2 * (y_true - dvalues) / outputs
        self.dinputs = self.dinputs / samples


# Define Accuracy metric
class Accuracy:
    def calculate(self, predictions, y):
        comparisons = self.compare(predictions, y)
        accuracy = np.mean(comparisons)
        return accuracy
    
    def compare(self, predictions, y):
        return predictions == y


class Neural_Network:
    def __init__(
        self, n_inputs_attr, n_hidden_layers, nodes_in_hidden_layers, n_outputs
    ):
        # using Layer class to create layers
        # nodes_in_hidden_layers is a list containing the number of nodes in each hidden layer
        self.layers = []
        for i in range(n_hidden_layers):
            if i == 0:
                self.layers.append(Layer(n_inputs_attr, nodes_in_hidden_layers[i]))
            else:
                self.layers.append(
                    Layer(nodes_in_hidden_layers[i - 1], nodes_in_hidden_layers[i])
                )
        if n_hidden_layers > 0:
            self.layers.append(Layer(nodes_in_hidden_layers[-1], n_outputs, 1))
        else:
            self.layers.append(Layer(n_inputs_attr, n_outputs, 1))

    def print_layer(self, layer):
        print("Layer Inputs")
        print(layer.inputs, layer.inputs.shape)
        print("Layer Weights")
        print(layer.weights, layer.weights.shape)
        print("Layer Biases")
        print(layer.biases, layer.biases.shape)
        print("Layer Output")
        print(layer.output, layer.output.shape)
        print("Layer Activation Output")
        print(layer.activation.output, layer.activation.output.shape)

    def print_layers(self):
        for layer in self.layers:
            # if layer an object of Layer class
            print("LAYER")
            if isinstance(layer, Layer):
                self.print_layer(layer)

    def forward(self, X):
        for layer in self.layers:
            layer.forward(X)
            X = layer.activation.output
        return X

    def backward(self, y_pred, y_true, learning_rate):
        output_error = y_true - y_pred.flatten()
        for layer in reversed(self.layers):
            # print("LAYER", self.layers.index(layer))
            # self.print_layer(layer)
            if layer == self.layers[-1]:
                layer.backward(output_error, 1)
            else:
                layer.backward(output_error)
            output_error = layer.activation.dinputs
        for layer in self.layers:
            layer.update(learning_rate)

    def train(self, X, y, epochs, learning_rate):
        batch_size = 8
        X_train_batches = [
            X[i : i + batch_size] for i in range(0, len(X), batch_size)
        ]
        y_train_batches = [
            y[i : i + batch_size] for i in range(0, len(y), batch_size)
        ]
        loss = MeanSquaredError()
        accuracy = Accuracy()
        for epoch in range(epochs):
            for X_batch, y_batch in zip(X_train_batches, y_train_batches):
                y_pred = self.forward(X_batch)
                # print("y_pred, y_baytch", y_pred, y_batch)
                loss_value = loss.calculate(y_pred, y_batch)
                accuracy_value = accuracy.calculate(y_pred, y_batch)
                print(
                    f"Epoch: {epoch}, Loss: {loss_value:.3f}, Accuracy: {accuracy_value:.3f}"
                )
                self.backward(y_pred, y_batch, learning_rate)

    def predict(self, X):
        return self.forward(X)


config = [[0, [0]], [1, [32]], [1, [64]], [1, [3]]]
# for config in configurations:
n_hidden_layers = config[3][0]
nodes_in_hidden_layers = config[3][1]
# Create a new model with the given configuration
model = Neural_Network(X_train.shape[1], n_hidden_layers, nodes_in_hidden_layers, 1)
# Train the model
model.train(X_train, y_train, 100, 0.001)
# Test the model
predictions = model.predict(X_test)
# store predictions and actual values in a file

Epoch: 0, Loss: 120.373, Accuracy: 0.000
Output Layer
dinputs [[-0.21610425 -0.40733397 -0.42389001]
 [-0.11423906 -0.21532871 -0.22408071]
 [-0.72506607 -1.36667388 -1.42222217]
 [-0.47047368 -0.8867938  -0.92283742]
 [-0.67409646 -1.2706015  -1.32224494]
 [-0.52168204 -0.98331622 -1.02328298]
 [-0.31796136 -0.59932398 -0.62368343]
 [-0.9288487  -1.75078289 -1.82194323]] (8, 3)
mask [[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]] (8, 3)
dinputs [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] (8, 3)
Hidden Layer


ValueError: operands could not be broadcast together with shapes (8,8) (8,3) 

In [ ]:
model = Neural_Network(X_train.shape[1], n_hidden_layers, nodes_in_hidden_layers, 1)

In [204]:
# neural_net.backward(y_pred, y_true, 0.01)
# round the predictions 
predictions = np.round(predictions)
a = Accuracy()
print(a.calculate(predictions, y_test))
combined_values = np.hstack((predictions.reshape(-1, 1), y_test.reshape(-1, 1)))

# Save the combined values to a text file
with open(f"output_{n_hidden_layers}_{nodes_in_hidden_layers}.txt", "w") as file:
    # save loss 
    # file.write(f"Loss: {loss_value:.3f}\n")
    for row in combined_values:
        file.write(f"{row[0]} {row[1]}\n")
# neural_net.print_layers()

0.10766408736063734
